In [3]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable
from arguments_dict import arguments_dict

executables = ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-imm', 'genmc-xmm']
executable_paths_dict = dict([(exe, Path("executables") / exe) for exe in executables])
tests = dict([(t, Path('data-structures') / t / 'variants' / (os.listdir(Path('data-structures') / t / 'variants')[0])) for t in os.listdir('data-structures') if os.path.isdir(Path('data-structures') / t)])
subprocess_timeout = 60 #s takes 10 mins with 60s timeout
test_arguments = {
    "spinlock": "N=4",
    "ttaslock": "N=4",
    "barrier": "NUMREADERS=5", 
    "mutex": "N=2",
    "fcombiner-async": "N_THREADS=2",
    "dq": "",
    "chase-lev": "CONFIG_DEQ_STEALERS=2",
    "twalock": "N=3",
    "ticketlock": "N=6",
    "stc": "N=6",
    "stc-opt": "",
    "linuxrwlocks": "CONFIG_RWLOCK_READERS=2 CONFIG_RWLOCK_WRITERS=3",
    "ms-queue": "CONFIG_QUEUE_READERS=1 CONFIG_QUEUE_WRITERS=2",
    "treiber-stack": "CONFIG_STACK_READERS=2 CONFIG_STACK_WRITERS=2",
    "mpmc-queue": "CONFIG_MPMC_READERS=3 CONFIG_MPMC_WRITERS=2",
    "buf_ring": "",
    "mcs_spinlock": "N=4",
    "seqlock": "CONFIG_SEQ_READERS=2 CONFIG_SEQ_WRITERS=2",
    "seqlock-atomic": "CONFIG_SEQ_READERS=2 CONFIG_SEQ_WRITERS=2",
    "barrier-bnd": "NUMREADERS=4",
    "mpmc-queue-bnd": "",
    "seqlock-bnd": "CONFIG_LOCK_READERS=0 CONFIG_LOCK_WRITERS=3 CONFIG_LOCK_RDWR=0",
    # linuxrwlocks-bnd segfaults
    # treiber-stack-bnd llvm15 crashes
    # qspinlock doesn't compile
    # qu doesn't terminate
}
final_tests = set([
    "mpmc-queue-bnd",
    "ticketlock",
    "chase-lev",
    "buf-ring",
    "dq",
    "stc",
    "linuxrwlocks",
    "twalock",
    "fcombiner-async",
    "mutex"
])
test_arguments = {key: value for key, value in test_arguments.items() if key in final_tests} 

class RunExecutableResult:
    class Timeout:
        pass
    class Ok:
        def __init__(self, output) -> None:
            super().__init__()
            self.output = output
    class Error:
        def __init__(self, retcode, output) -> None:
            super().__init__()
            self.retcode = retcode
            self.output = output

def run_executable(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    try:
        result = subprocess.run([exe_path, *args, test_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=subprocess_timeout)
        result_stdout = result.stdout.decode()
        result_stderr = result.stderr.decode()
        if result.returncode == 0:
            return RunExecutableResult.Ok(result_stdout + result_stderr)
        else:
            return RunExecutableResult.Error(result.returncode, result_stdout + result_stderr)
    except subprocess.TimeoutExpired:
        return RunExecutableResult.Timeout()
    

In [4]:
def get_execution_time(string: str) -> Optional[float]:
    try:
        pattern = "Total wall-clock time: "
        time = string.split(pattern)[1].split('s\n')[0]
        return float(time)
    except:
        return None
assert(get_execution_time(
"""
No errors were detected.
Number of complete executions explored: 3
Total wall-clock time: 0.05s
""") == 0.05)

def get_number_executions(string: str) -> Optional[int]:
    pattern = "Number of complete executions explored: "
    i = string.index(pattern) + len(pattern)
    number_str = ""
    while i < len(string) and string[i].isdigit():
        number_str += string[i]
        i += 1
    try:
        return int(number_str)
    except:
        return None

assert(get_number_executions("abc 123 Number of complete executions explored: 123456") == 123456)

def get_duplicate_executions(string: str) -> Optional[int]:
    try:
        pattern = "Number of duplicate executions explored: "
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_duplicate_executions("abc 123 Number of duplicate executions explored: 123456") == 123456)

def get_number_lb_races(string: str) -> Optional[int]:
    pattern = "Number of load buffering races explored: "
    try:
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_number_lb_races("abc 123 Number of load buffering races explored: 123456") == 123456)

class ResultItem:
    def __init__(self, execs, exe_time, dups, lb_races) -> None:
        self.execs = execs
        self.exe_time = exe_time
        self.dups = dups
        self.lb_races = lb_races

def run_and_get_results(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    res = run_executable(exe_path, args, test_path)
    if isinstance(res, RunExecutableResult.Ok):
        execs = get_number_executions(res.output)
        exe_time = get_execution_time(res.output)
        dups = get_duplicate_executions(res.output)
        lb_races = get_number_lb_races(res.output)
        if execs is None:
            # print(f"could not get number of executions in {res.output}")
            execs = 0
        if dups is None:
            dups = 0
        if exe_time is None:
            # print(f"could not get duplicate executions in {res.output}")
            exe_time = 0
        return RunExecutableResult.Ok(ResultItem(execs, exe_time, dups, lb_races))
    else:
        return res

In [5]:
timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'
rows = [["Test Name", "Old GenMC Execs",  "Old GenMC Duplicates", "Old GenMC Time", "WMC Execs", "WMC Duplicates", "WMC Time", "GenMC Execs", "GenMC Duplicates", "GenMC Time", "GenMC IMM Execs", "GenMC IMM Duplicates", "GenMC IMM Time", "XMM Execs", "XMM Duplicates", "XMM Time", "LB races"]]
assert(executables == ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-imm', 'genmc-xmm'])
for test_name in test_arguments.keys():
    row = [f"{test_name}"]
    for exe in executables:
        if test_name in test_arguments:
            args = arguments_dict[exe] + ["--"] + [f"-D{a}" for a in test_arguments[test_name].split()]
        else:
            args = arguments_dict[exe]
        res = run_and_get_results(executable_paths_dict[exe], args, tests[test_name])
        if isinstance(res, RunExecutableResult.Ok):
            execs = res.output.execs
            exe_time = res.output.exe_time      
            dups = res.output.dups          
            row += [execs, dups, f"{exe_time:.2f}s"]
            if res.output.lb_races is not None:
                row += [res.output.lb_races]
        elif isinstance(res, RunExecutableResult.Timeout):
            print(f"{exe} {test_name} timed out")
            row += [timeout] * 3
        elif isinstance(res, RunExecutableResult.Error):
            print(f"{exe} {test_name} error {res.retcode}:\n{res.output[:1000]}")
            # row += [res.output[:60] if res.output != "" else f"error code {res.retcode}"] * 2
            row = None
            break
        else:
            assert(0)
    
    rows.append(row) if row is not None else None

# assert(len(rows) == 16)
print("rows has len", len(rows))


genmc-old fcombiner-async timed out
genmc-wkmo fcombiner-async timed out
genmc-old linuxrwlocks timed out
genmc-wkmo linuxrwlocks timed out
rows has len 10


In [6]:
data = rows[1:]
data.sort(key=lambda x: x[-1] if isinstance(x[-1], int) else -1, reverse=True)

for i in range(len(data)):
    data[i][0] = data[i][0].replace("_", "-")

data = [d for d in data if len(d) == len(rows[0])]

data = [d for d in data if d[-1] > 0] # remove tests without LB races

rows = [rows[0]] + data

In [7]:
titles = ["Test Name", "Old GenMC$_\\RCMM$", "WMC", "GenMC$_\\RCMM$", "GenMC$_\\IMM$", "GenMC-XMM", "LB Races"]
header = [[" ", "\\multicolumn{6}{c|}{Number of Executions}", "", "", "", "", ""], titles]

exec_rows = header + [[row[0]] + row[1:-1][0::3] + [row[-1]] for row in rows[1:]] 

for i in range(2, len(exec_rows)):
    execs = exec_rows[i][1]
    if exec_rows[i][5] != exec_rows[i][3]:
        exec_rows[i][0] = f"\\textcolor{{blue}}{{{exec_rows[i][0]}}}"
    elif any(p != execs for p in exec_rows[i][1:-1]):
        exec_rows[i][0] = f"\\textcolor{{red}}{{{exec_rows[i][0]}}}"

table_2 = Texttable()
table_2.set_cols_align(["l"] + ["r" for i in range(len(exec_rows[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(exec_rows[0]))])
table_2.add_rows(exec_rows)

print(table_2.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
|           | \multicol |          |          |          |          |          |
|           | umn{6}{c| |          |          |          |          |          |
|           | }{Number  |          |          |          |          |          |
|           | of Execut |          |          |          |          |          |
|           |   ions}   |          |          |          |          |          |
+===========+===========+==========+==========+==========+==========+==========+
| Test Name | Old GenMC |      WMC | GenMC$_\ | GenMC$_\ |   GenMC- | LB Races |
|           |  $_\RCMM$ |          |    RCMM$ |     IMM$ |      XMM |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| mpmc-     |     15752 |    15752 |    15752 |    15752 |    15752 |    24240 |
| queue-bnd |           |          |          |          |          |          |
+-----------+-----------+---

In [8]:
header = [[" ", "\\multicolumn{6}{c|}{Execution Time}", "", "", "", "", ""], titles]

time_rows = header + [[row[0]] + row[3:-1][0::3] + [row[-1]] for row in rows[1:]]

for x in time_rows:
    assert(len(x) == len(time_rows[0]))

for i in range(2, len(time_rows)):
    time_rows[i][0] = exec_rows[i][0]

table_1 = Texttable()
table_1.set_cols_align(["l"] + ["r" for i in range(len(time_rows[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(time_rows[0]))])
table_1.add_rows(time_rows)

print(table_1.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
|           | \multicol |          |          |          |          |          |
|           | umn{6}{c| |          |          |          |          |          |
|           | }{Executi |          |          |          |          |          |
|           | on Time}  |          |          |          |          |          |
+===========+===========+==========+==========+==========+==========+==========+
| Test Name | Old GenMC |      WMC | GenMC$_\ | GenMC$_\ |   GenMC- | LB Races |
|           |  $_\RCMM$ |          |    RCMM$ |     IMM$ |      XMM |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| mpmc-     |     3.11s |    3.35s |    3.77s |    7.43s |   10.12s |    24240 |
| queue-bnd |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| ticketloc |     3.78s |   

In [9]:
header = [[" ", "\\multicolumn{6}{c|}{Number of Duplicates}", "", "", "", "", ""], titles]

dups_rows = header + [[row[0]] + row[2:-1][0::3] + [row[-1]] for row in rows[1:]]

table_3 = Texttable()
table_3.set_cols_align(["l"] + ["r" for i in range(len(dups_rows[0]) - 1)])
table_3.set_cols_valign(["m" for i in range(len(dups_rows[0]))])
table_3.add_rows(dups_rows)



In [10]:
def adjust_header(table: str) -> str:
    return table.replace("\\multicolumn{6}{c|}{Number of Executions} &  &  &  &  &", "\\multicolumn{6}{c|}{Number of Executions}").replace("\\multicolumn{6}{c|}{Execution Time} &  &  &  &  &", "\\multicolumn{6}{c|}{Execution Time}")

def adjust_width(table: str) -> str:
    return table.replace("\\begin{center}", "\\begin{center} \\resizebox{\\textwidth}{!}{").replace("\\end{tabular}", "\\end{tabular}}")

caption = f'on data structure benchmarks run with the version of GenMC that WMC is based on ("Old GenMC$_\\RCMM$"), WMC, GenMC$_\\RCMM$, GenMC$_\\IMM$, and GenMC-XMM. Timeout was set to {subprocess_timeout}s'
table_2_code = latextable.draw_latex(table_2, position='H', caption=f"Number of executions explored" + caption, label="table:data-structures-benchmarks-execs")
table_2_code = adjust_width(adjust_header(table_2_code))
print(table_2_code)

table_1_code = latextable.draw_latex(table_1, position='H', caption=f"Execution time" + caption, label="table:data-structures-benchmarks-time")
table_1_code = adjust_width(adjust_header(table_1_code))
print(table_1_code)

# print(latextable.draw_latex(table_3, position='H', caption=f"Duplicate executions on data structure benchmarks run with the version of GenMC that WMC is based on, WMC, GenMC, and GenMC-XMM. Timeout was set to {subprocess_timeout}s", label="table:data-structures-benchmarks-dups"))

\begin{table}[H]
	\begin{center} \resizebox{\textwidth}{!}{
		\begin{tabular}{|l|r|r|r|r|r|r|}
			\hline
			  & \multicolumn{6}{c|}{Number of Executions}  \\
			\hline
			Test Name & Old GenMC$_\RCMM$ & WMC & GenMC$_\RCMM$ & GenMC$_\IMM$ & GenMC-XMM & LB Races \\
			\hline
			mpmc-queue-bnd & 15752 & 15752 & 15752 & 15752 & 15752 & 24240 \\
			\hline
			ticketlock & 720 & 720 & 720 & 720 & 720 & 10800 \\
			\hline
			\textcolor{blue}{chase-lev} & 3809 & 3975 & 3639 & 3639 & 3757 & 6498 \\
			\hline
			stc & 183 & 183 & 183 & 183 & 183 & 1515 \\
			\hline
			\textcolor{blue}{dq} & 1802 & 1802 & 1924 & 1924 & 1931 & 1485 \\
			\hline
			\textcolor{red}{linuxrwlocks} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} & 216 & 216 & 216 & 384 \\
			\hline
			twalock & 96 & 96 & 96 & 96 & 96 & 288 \\
			\hline
			\textcolor{red}{fcombiner-async} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} & 24 & 24 & 24 & 32 \\
			\hline
			mutex & 12 & 12

In [11]:
print(table_1.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
|           | \multicol |          |          |          |          |          |
|           | umn{6}{c| |          |          |          |          |          |
|           | }{Executi |          |          |          |          |          |
|           | on Time}  |          |          |          |          |          |
+===========+===========+==========+==========+==========+==========+==========+
| Test Name | Old GenMC |      WMC | GenMC$_\ | GenMC$_\ |   GenMC- | LB Races |
|           |  $_\RCMM$ |          |    RCMM$ |     IMM$ |      XMM |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| mpmc-     |     3.11s |    3.35s |    3.77s |    7.43s |   10.12s |    24240 |
| queue-bnd |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| ticketloc |     3.78s |   

In [12]:
print(table_2.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
|           | \multicol |          |          |          |          |          |
|           | umn{6}{c| |          |          |          |          |          |
|           | }{Number  |          |          |          |          |          |
|           | of Execut |          |          |          |          |          |
|           |   ions}   |          |          |          |          |          |
+===========+===========+==========+==========+==========+==========+==========+
| Test Name | Old GenMC |      WMC | GenMC$_\ | GenMC$_\ |   GenMC- | LB Races |
|           |  $_\RCMM$ |          |    RCMM$ |     IMM$ |      XMM |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| mpmc-     |     15752 |    15752 |    15752 |    15752 |    15752 |    24240 |
| queue-bnd |           |          |          |          |          |          |
+-----------+-----------+---

In [13]:
print(table_3.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
|           | \multicol |          |          |          |          |          |
|           | umn{6}{c| |          |          |          |          |          |
|           | }{Number  |          |          |          |          |          |
|           | of Duplic |          |          |          |          |          |
|           |   ates}   |          |          |          |          |          |
+===========+===========+==========+==========+==========+==========+==========+
| Test Name | Old GenMC |      WMC | GenMC$_\ | GenMC$_\ |   GenMC- | LB Races |
|           |  $_\RCMM$ |          |    RCMM$ |     IMM$ |      XMM |          |
+-----------+-----------+----------+----------+----------+----------+----------+
| mpmc-     |         0 |        0 |        0 |        0 |        0 |    24240 |
| queue-bnd |           |          |          |          |          |          |
+-----------+-----------+---